In [1]:
import sys 
print('\n'.join(sys.path))
sys.path.append('/home/deeplearning/.local/lib/python3.7/site-packages')
print('\n')
print('\n'.join(sys.path))

/home/deeplearning/Dicom-To-CNN
/tmp/0ed42a98-b821-4fdf-9a65-921f51808625
/home/deeplearning/.vscode/extensions/ms-python.python-2020.8.105369/pythonFiles
/home/deeplearning/.vscode/extensions/ms-python.python-2020.8.105369/pythonFiles/lib/python
/usr/lib/python38.zip
/usr/lib/python3.8
/usr/lib/python3.8/lib-dynload

/home/deeplearning/.local/lib/python3.8/site-packages
/usr/local/lib/python3.8/dist-packages
/usr/lib/python3/dist-packages
/home/deeplearning/.local/lib/python3.8/site-packages/IPython/extensions
/home/deeplearning/.ipython


/home/deeplearning/Dicom-To-CNN
/tmp/0ed42a98-b821-4fdf-9a65-921f51808625
/home/deeplearning/.vscode/extensions/ms-python.python-2020.8.105369/pythonFiles
/home/deeplearning/.vscode/extensions/ms-python.python-2020.8.105369/pythonFiles/lib/python
/usr/lib/python38.zip
/usr/lib/python3.8
/usr/lib/python3.8/lib-dynload

/home/deeplearning/.local/lib/python3.8/site-packages
/usr/local/lib/python3.8/dist-packages
/usr/lib/python3/dist-packages
/home/dee

In [2]:
from library_dicom.dicom_processor.tools.flip import *
from library_dicom.dicom_processor.tools.create_mip import *

In [3]:
from radiomics import featureextractor

Error loading C extensions
Traceback (most recent call last):
  File "/home/deeplearning/.local/lib/python3.7/site-packages/radiomics/__init__.py", line 283, in <module>
    from radiomics import _cmatrices as cMatrices  # noqa: F401
ImportError: cannot import name '_cmatrices' from partially initialized module 'radiomics' (most likely due to a circular import) (/home/deeplearning/.local/lib/python3.7/site-packages/radiomics/__init__.py)


ImportError: cannot import name '_cmatrices' from partially initialized module 'radiomics' (most likely due to a circular import) (/home/deeplearning/.local/lib/python3.7/site-packages/radiomics/__init__.py)

In [2]:

csv_flip_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/FLIP_PET0_NIFTI.csv'

with open(csv_flip_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne


In [3]:
csv_data[0]

['CR,014',
 '2017014',
 '1.2.250.1.38.2.1.102.140724093005.103468016',
 'pet0',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_CT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_PT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_mask.nii']

In [4]:
for serie in csv_data:
    print(csv_data.index(serie))
    roi, suv_max = get_suv_max_value(serie[6], serie[5])
    serie.append(roi)
    serie.append(suv_max)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [5]:
csv_data[0]

['CR,014',
 '2017014',
 '1.2.250.1.38.2.1.102.140724093005.103468016',
 'pet0',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_CT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_PT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/CR,014/pet0/1.2.250.1.38.2.1.102.140724093005.103468016_nifti_mask.nii',
 19,
 9.66]

In [7]:
nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'

with open(os.path.join(nifti_directory, 'FLIP_SUV_MAX_BIGGER_ROI.csv'), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT_NAME", "PATIENT_ID", "STUDY_UID", "PET", "NIFTI_CT" "NIFTI_PET", "NIFTI_MASK", "ROI", "SUV_MAX"])
    for serie in csv_data: 
        csv_writer.writerow([serie[0], serie[1], serie[2], serie[3], serie[4], serie[5], serie[6], serie[7]+1, serie[8]])